# L2: DSPy Programming - Signatures and Modules

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [24]:
import sys

!{sys.executable} -m pip install --upgrade litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 41.9 MB/s eta 0:00:0000:01
  Attempting uninstall: litellm
    Found existing installation: litellm 1.61.13
    Uninstalling litellm-1.61.13:
      Successfully uninstalled litellm-1.61.13


In [32]:
pip show litellm

Name: litellm
Version: 1.72.1
Summary: Library to easily interface with LLM API providers
Home-page: https://litellm.ai
Author: BerriAI
Author-email: 
License: MIT
Location: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages
Requires: aiohttp, click, httpx, importlib-metadata, jinja2, jsonschema, openai, pydantic, python-dotenv, tiktoken, tokenizers
Required-by: crewai, dspy
Note: you may need to restart the kernel to use updated packages.


In [25]:
from helper import get_openai_api_key
openai_api_key = get_openai_api_key()

import os

os.environ["OPENAI_API_KEY"]  = get_openai_api_key()

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.</p>

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

### Set up API key

In [1]:
from helper import get_openai_api_key
openai_api_key = get_openai_api_key()

import os

os.environ["OPENAI_API_KEY"] = get_openai_api_key()

### Configure the LM

In [18]:
import dspy
dspy.settings.configure(lm=dspy.LM("openai/gpt-4o-mini"))

## Use DSPy built-in Module to Build a Sentiment Classifier

In [19]:
class SentimentClassifier(dspy.Signature):
    """Classify the sentiment of a text."""

    text: str = dspy.InputField(desc="input text to classify sentiment")
    sentiment: int = dspy.OutputField(
        desc="sentiment, the higher the more positive", ge=0, le=10
    )

In [20]:
str_signature = dspy.make_signature("text -> sentiment")

### Create a Module to Interact with the LM

In [21]:
predict = dspy.Predict(SentimentClassifier) 

In [22]:
output = predict(text="I am feeling pretty happy!")
print(output)

Prediction(
    sentiment=8
)


In [23]:
print(f"The sentiment is: {output.sentiment}")
print(f"The sentiment is: {output['sentiment']}")

The sentiment is: 8
The sentiment is: 8


In [24]:
dspy.configure(lm=dspy.LM("openai/gpt-4o"))
print(predict(text="I am feeling pretty happy!"))

Prediction(
    sentiment=4
)


In [25]:
dspy.configure(lm=dspy.LM("openai/gpt-4o-mini"))

### Wait, Where is My Prompt? 

In [26]:
dspy.inspect_history(n=1)





[2025-06-05T12:40:50.180584]

System message:

Your input fields are:
1. `text` (str): input text to classify sentiment

Your output fields are:
1. `sentiment` (int): sentiment, the higher the more positive

All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## text ## ]]
{text}

Outputs will be a JSON object with the following fields.

{
  "sentiment": "{sentiment}        # note: the value you produce must be a single int value"
}

In adhering to this structure, your objective is: 
        Classify the sentiment of a text.


User message:

[[ ## text ## ]]
I am feeling pretty happy!

Respond with a JSON object in the following order of fields: `sentiment` (must be formatted as a valid Python int).


Response:

{"sentiment":4}







### Try a Different Built-in Module

In [27]:
cot = dspy.ChainOfThought(SentimentClassifier)

output = cot(text="I am feeling pretty happy!")
print(output)

Prediction(
    reasoning="Reasoning: Let's think step by step in order to analyze the text. The phrase 'I am feeling pretty happy!' expresses a positive emotion. The use of the word 'happy' indicates a strong positive sentiment. Therefore, the overall sentiment of the text is positive.",
    sentiment=8
)


In [28]:
dspy.inspect_history(n=1)





[2025-06-05T12:40:55.439895]

System message:

Your input fields are:
1. `text` (str): input text to classify sentiment

Your output fields are:
1. `reasoning` (str)
2. `sentiment` (int): sentiment, the higher the more positive

All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## text ## ]]
{text}

Outputs will be a JSON object with the following fields.

{
  "reasoning": "{reasoning}",
  "sentiment": "{sentiment}        # note: the value you produce must be a single int value"
}

In adhering to this structure, your objective is: 
        Classify the sentiment of a text.


User message:

[[ ## text ## ]]
I am feeling pretty happy!

Respond with a JSON object in the following order of fields: `reasoning`, then `sentiment` (must be formatted as a valid Python int).


Response:

{"reasoning":"Reasoning: Let's think step by step in order to analyze the text. The phrase 'I am feeling pretty h

### Use a Different Adapter

In [29]:
dspy.configure(adapter=dspy.JSONAdapter())

In [30]:
print(cot(text="I am feeling pretty happy!"))
dspy.inspect_history(n=1)

Prediction(
    reasoning="Reasoning: Let's think step by step in order to analyze the text. The phrase 'I am feeling pretty happy!' expresses a positive emotion. The use of the word 'happy' indicates a strong positive sentiment. Therefore, the overall sentiment of the text is positive.",
    sentiment=8
)




[2025-06-05T12:41:00.645861]

System message:

Your input fields are:
1. `text` (str): input text to classify sentiment

Your output fields are:
1. `reasoning` (str)
2. `sentiment` (int): sentiment, the higher the more positive

All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## text ## ]]
{text}

Outputs will be a JSON object with the following fields.

{
  "reasoning": "{reasoning}",
  "sentiment": "{sentiment}        # note: the value you produce must be a single int value"
}

In adhering to this structure, your objective is: 
        Classify the sentiment of a text.


User message

## Build a Program with Custom Module

In [31]:
class QuestionGenerator(dspy.Signature):
    """Generate a yes or no question in order to guess the celebrity name in users' mind. You can ask in general or directly guess the name if you think the signal is enough. You should never ask the same question in the past_questions."""
    past_questions: list[str] = dspy.InputField(desc="past questions asked")
    past_answers: list[bool] = dspy.InputField(desc="past answers")
    new_question: str = dspy.OutputField(desc="new question that can help narrow down the celebrity name")
    guess_made: bool = dspy.OutputField(desc="If the new_question is the celebrity name guess, set to True, if it is still a general question set to False")


class Reflection(dspy.Signature):
    """Provide reflection on the guessing process"""
    correct_celebrity_name: str = dspy.InputField(desc="the celebrity name in user's mind")
    final_guessor_question: str = dspy.InputField(desc="the final guess or question LM made")
    past_questions: list[str] = dspy.InputField(desc="past questions asked")
    past_answers: list[bool] = dspy.InputField(desc="past answers")

    reflection: str = dspy.OutputField(
        desc="reflection on the guessing process, including what was done well and what can be improved"
    )

def ask(prompt, valid_responses=("y", "n")):
    while True:
        response = input(f"{prompt} ({'/'.join(valid_responses)}): ").strip().lower()
        if response in valid_responses:
            return response
        print(f"Please enter one of: {', '.join(valid_responses)}")

class CelebrityGuess(dspy.Module):
    def __init__(self, max_tries=10):
        super().__init__()

        self.question_generator = dspy.ChainOfThought(QuestionGenerator)
        self.reflection = dspy.ChainOfThought(Reflection)

        self.max_tries = 20

    def forward(self):
        celebrity_name = input("Please think of a celebrity name, once you are ready, type the name and press enter...")
        past_questions = []
        past_answers = []

        correct_guess = False

        for i in range(self.max_tries):
            question = self.question_generator(
                past_questions=past_questions,
                past_answers=past_answers,
            )
            answer = ask(f"{question.new_question}").lower() == "y"
            past_questions.append(question.new_question)
            past_answers.append(answer)

            if question.guess_made and answer:
                correct_guess = True
                break

        if correct_guess:
            print("Yay! I got it right!")
        else:
            print("Oops, I couldn't guess it right.")

        reflection = self.reflection(
            correct_celebrity_name=celebrity_name,
            final_guessor_question=question.new_question,
            past_questions=past_questions,
            past_answers=past_answers,
        )
        print(reflection.reflection)


In [32]:
celebrity_guess = CelebrityGuess()

In [33]:
celebrity_guess()

Please think of a celebrity name, once you are ready, type the name and press enter... Donald Trump
New Question: Is the celebrity you are thinking of male? (y/n):  y
New Question: Is the celebrity you are thinking of known for acting? (y/n):  n
New Question: Is the celebrity you are thinking of known for music? (y/n):  n
New Question: Is the celebrity you are thinking of known for sports? (y/n):  n
New Question: Is the celebrity you are thinking of known for their work in politics? (y/n):  y
New Question: Is the celebrity you are thinking of currently active in politics? (y/n):  y
New Question: Is the celebrity you are thinking of a current or former president? (y/n):  y
New Question: Is the celebrity you are thinking of still alive? (y/n):  y
New Question: Is the celebrity you are thinking of a Democrat? (y/n):  n
New Question: Is the celebrity you are thinking of known for a significant political event or action during their presidency? (y/n):  y
New Question: Did the celebrity you 

Oops, I couldn't guess it right.
Reflection: The guessing process was quite thorough, as it effectively narrowed down the options through a series of targeted questions. The questions about political affiliation and significant events during the presidency were particularly useful. However, the final question could have been framed differently to avoid confusion, as it directly asked about the current president, which led to a false conclusion. In future iterations, it might be beneficial to clarify the time frame of the presidency in the questions to avoid such pitfalls.


## Save and Load

In [34]:
celebrity_guess.save("dspy_program/celebrity.json", save_program=False)

In [35]:
celebrity_guess.load("dspy_program/celebrity.json")

In [36]:
celebrity_guess.save("dspy_program/celebrity/", save_program=True)

In [37]:
loaded = dspy.load("dspy_program/celebrity/")

In [ ]:
loaded()